- Image batches are commonly represented by a 4-D array with shape (batch_size, num_channels, width, height)
- mnist images are grayscale, the input is `(batch_size, 1, 28, 28)`
- When feeding training examples, it is critical that we don’t feed samples with the same label in succession. Doing so can slow down training. Data iterators take care of this by randomly shuffling the inputs.
- MLPs contains several fully connected layers. A fully connected layer or FC layer for short, is one where each neuron in the layer is connected to every neuron in its preceding layer. From a linear algebra perspective, an FC layer applies an affine transform to the n x m input matrix X and outputs a matrix Y of size n x k, where k is the number of neurons in the FC layer. k is also referred to as the hidden size. The output Y is computed according to the equation Y = W X + b. The FC layer has two learnable parameters, the m x k weight matrix W and the m x 1 bias vector b.

In [2]:
import mxnet as mx
mnist = mx.test_utils.get_mnist()

/home/zhenglai/repo/incubator-mxnet/python/mxnet/test_utils.py:1431: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(flbl.read(), dtype=np.int8)
/home/zhenglai/repo/incubator-mxnet/python/mxnet/test_utils.py:1434: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  image = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(label), rows, cols)


In [57]:
batch_size = 2 # 100
train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

In [58]:
data = mx.sym.var('data')
# Flatten the data from 4-D shape into 2-D (batch_size, num_channel*width*height)
data = mx.sym.flatten(data=data)

In [59]:
fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type='relu')
fc2 = mx.sym.FullyConnected(data=act1, num_hidden=64)
act2 = mx.sym.Activation(data=fc2, act_type='relu')
fc3 = mx.sym.FullyConnected(data=act2, num_hidden=10)
mlp = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

In [60]:
import numpy
#@register
#@alias('ce')
class CrossEntropy(mx.metric.EvalMetric):
    def __init__(self, eps=1e-12, name='cross-entropy',
                 output_names=None, label_names=None):
        super(CrossEntropy, self).__init__(
            name, eps=eps,
            output_names=output_names, label_names=label_names)
        self.eps = eps

    def update(self, labels, preds):
        # check_label_shapes(labels, preds)
        print('labels={}'.format(labels))
        print('preds={}'.format(preds))
        for label, pred in zip(labels, preds):
            label = label.asnumpy()
            pred = pred.asnumpy()
            print('label={}'.format(label))
            print('pred={}'.format(pred))
            label = label.ravel()
            assert label.shape[0] == pred.shape[0]

            prob = pred[numpy.arange(label.shape[0]), numpy.int64(label)]
            self.sum_metric += (-numpy.log(prob + self.eps)).sum()
            self.num_inst += label.shape[0]



In [61]:
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on CPU
mlp_model = mx.mod.Module(symbol=mlp, context=mx.cpu())
mlp_model.fit(train_iter,  # train data
              eval_data=val_iter,  # validation data
              optimizer='sgd',  # use SGD to train
              optimizer_params={'learning_rate':0.1},  # use fixed learning rate
              eval_metric= [CrossEntropy()], # 'acc',  # report accuracy during training
              batch_end_callback = mx.callback.Speedometer(batch_size, 100), # output progress for each 100 data batches
              num_epoch=10)  # train for at most 10 dataset passes

labels=[
[6. 6.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.09999979 0.0999874  0.10002475 0.09999491 0.10000254 0.09999043
  0.10001322 0.09998852 0.09999517 0.10000326]
 [0.10000587 0.09999114 0.10002345 0.09998234 0.10001592 0.09999555
  0.10001698 0.09998593 0.09999037 0.09999247]]
<NDArray 2x10 @cpu(0)>]
label=[6. 6.]
pred=[[0.09999979 0.0999874  0.10002475 0.09999491 0.10000254 0.09999043
  0.10001322 0.09998852 0.09999517 0.10000326]
 [0.10000587 0.09999114 0.10002345 0.09998234 0.10001592 0.09999555
  0.10001698 0.09998593 0.09999037 0.09999247]]
labels=[
[3. 1.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.09896198 0.09894928 0.09897467 0.09895441 0.0989591  0.09894476
  0.10940453 0.09894499 0.09895106 0.09895527]
 [0.09896223 0.09895806 0.09896589 0.09895356 0.09896521 0.09895048
  0.10938646 0.09894519 0.09895179 0.09896115]]
<NDArray 2x10 @cpu(0)>]
label=[3. 1.]
pred=[[0.09896198 0.09894928 0.09897467 0.09895441 0.0989591  0.09894476
  0.10940453 0.09894499 0.09895106 0.09895527]
 [0.09896223 0

INFO:root:Epoch[0] Batch [100]	Speed: 615.45 samples/sec	cross-entropy=2.308332


preds=[
[[0.08606645 0.10312925 0.09500643 0.08555174 0.08543436 0.10593444
  0.09916592 0.12969993 0.10345749 0.10655395]
 [0.08606302 0.10311531 0.09500829 0.08554695 0.08542976 0.1059358
  0.09918337 0.12969792 0.10346162 0.10655794]]
<NDArray 2x10 @cpu(0)>]
label=[1. 6.]
pred=[[0.08606645 0.10312925 0.09500643 0.08555174 0.08543436 0.10593444
  0.09916592 0.12969993 0.10345749 0.10655395]
 [0.08606302 0.10311531 0.09500829 0.08554695 0.08542976 0.1059358
  0.09918337 0.12969792 0.10346162 0.10655794]]
labels=[
[5. 8.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.08531524 0.10727561 0.094096   0.08480251 0.084692   0.10480019
  0.10323021 0.12799895 0.10237966 0.10540958]
 [0.08531069 0.10728138 0.09410085 0.08480553 0.08469199 0.10480297
  0.1032159  0.12800659 0.10237325 0.10541081]]
<NDArray 2x10 @cpu(0)>]
label=[5. 8.]
pred=[[0.08531524 0.10727561 0.094096   0.08480251 0.084692   0.10480019
  0.10323021 0.12799895 0.10237966 0.10540958]
 [0.08531069 0.10728138 0.09410085 0.08480553 0.084691

preds=[
[[0.09987003 0.11325121 0.09069921 0.09322055 0.09562507 0.08435968
  0.09824024 0.08514275 0.10607446 0.13351679]
 [0.099867   0.11324134 0.09070262 0.09322251 0.09563476 0.08435841
  0.09823417 0.08513018 0.10608201 0.133527  ]]
<NDArray 2x10 @cpu(0)>]
label=[3. 3.]
pred=[[0.09987003 0.11325121 0.09069921 0.09322055 0.09562507 0.08435968
  0.09824024 0.08514275 0.10607446 0.13351679]
 [0.099867   0.11324134 0.09070262 0.09322251 0.09563476 0.08435841
  0.09823417 0.08513018 0.10608201 0.133527  ]]
labels=[
[7. 3.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.09891079 0.11201742 0.08991297 0.10212421 0.09476425 0.08368981
  0.09731015 0.08445716 0.10500876 0.13180447]
 [0.09890398 0.11202256 0.08991531 0.10213163 0.09476157 0.08369029
  0.09730789 0.08444809 0.10501359 0.1318051 ]]
<NDArray 2x10 @cpu(0)>]
label=[7. 3.]
pred=[[0.09891079 0.11201742 0.08991297 0.10212421 0.09476425 0.08368981
  0.09731015 0.08445716 0.10500876 0.13180447]
 [0.09890398 0.11202256 0.08991531 0.10213163 0.0947

INFO:root:Epoch[0] Batch [200]	Speed: 607.16 samples/sec	cross-entropy=2.320872


...

<NDArray 2x10 @cpu(0)>]
label=[6. 2.]
pred=[[0.0793925  0.07452156 0.12725393 0.07203496 0.07684875 0.11009489
  0.13500638 0.07654308 0.16609813 0.08220588]
 [0.078293   0.07830174 0.124718   0.07283779 0.07823922 0.11046661
  0.12952332 0.08036517 0.16186897 0.08538613]]
labels=[
[0. 1.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.07987151 0.06081851 0.15283933 0.06713909 0.06782357 0.10267899
  0.17174591 0.06144687 0.16767795 0.06795832]
 [0.07011959 0.09871714 0.11895323 0.07508796 0.08286646 0.10880429
  0.11366804 0.09663446 0.13684748 0.09830134]]
<NDArray 2x10 @cpu(0)>]
label=[0. 1.]
pred=[[0.07987151 0.06081851 0.15283933 0.06713909 0.06782357 0.10267899
  0.17174591 0.06144687 0.16767795 0.06795832]
 [0.07011959 0.09871714 0.11895323 0.07508796 0.08286646 0.10880429
  0.11366804 0.09663446 0.13684748 0.09830134]]
labels=[
[7. 1.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.07104696 0.10703924 0.10581446 0.07178041 0.08834682 0.10565718
  0.10103713 0.11216342 0.12681568 0.11029866]
 [0.07197141 0

INFO:root:Epoch[0] Batch [700]	Speed: 538.31 samples/sec	cross-entropy=2.116469


preds=[
[[4.7888927e-02 3.3289049e-04 2.3892224e-02 2.1007848e-03 3.6731310e-04
  3.4096048e-03 9.2000443e-01 1.1557271e-04 1.7286708e-03 1.5958284e-04]
 [2.4185633e-02 2.8855522e-05 8.2411710e-03 3.7649056e-04 5.0099909e-05
  6.8856869e-04 9.6610826e-01 1.0105621e-05 2.9463918e-04 1.6140761e-05]]
<NDArray 2x10 @cpu(0)>]
label=[8. 6.]
pred=[[4.7888927e-02 3.3289049e-04 2.3892224e-02 2.1007848e-03 3.6731310e-04
  3.4096048e-03 9.2000443e-01 1.1557271e-04 1.7286708e-03 1.5958284e-04]
 [2.4185633e-02 2.8855522e-05 8.2411710e-03 3.7649056e-04 5.0099909e-05
  6.8856869e-04 9.6610826e-01 1.0105621e-05 2.9463918e-04 1.6140761e-05]]
labels=[
[0. 6.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.14455965 0.05181303 0.14957224 0.06475481 0.03562542 0.09250856
  0.22826679 0.03512737 0.16364881 0.03412334]
 [0.1503531  0.03987405 0.15272751 0.05707319 0.03137778 0.08357647
  0.26388413 0.02776435 0.16516899 0.02820048]]
<NDArray 2x10 @cpu(0)>]
label=[0. 6.]
pred=[[0.14455965 0.05181303 0.14957224 0.06475481 0

preds=[
[[0.7596476  0.00232953 0.02792564 0.02415129 0.01211177 0.04823139
  0.10251645 0.00118524 0.02053311 0.00136801]
 [0.7527687  0.00295269 0.02903731 0.026955   0.01449914 0.05447704
  0.09299619 0.00170627 0.02274165 0.00186599]]
<NDArray 2x10 @cpu(0)>]
label=[6. 0.]
pred=[[0.7596476  0.00232953 0.02792564 0.02415129 0.01211177 0.04823139
  0.10251645 0.00118524 0.02053311 0.00136801]
 [0.7527687  0.00295269 0.02903731 0.026955   0.01449914 0.05447704
  0.09299619 0.00170627 0.02274165 0.00186599]]
labels=[
[8. 7.]
<NDArray 2 @cpu(0)>]
preds=[
[[0.09652285 0.12295252 0.09285913 0.09582098 0.08762737 0.13091344
  0.10733996 0.08628499 0.10416031 0.0755185 ]
 [0.02610867 0.02668229 0.02367894 0.03069979 0.31720483 0.03578674
  0.01549969 0.2726093  0.05756195 0.1941678 ]]
<NDArray 2x10 @cpu(0)>]
label=[8. 7.]
pred=[[0.09652285 0.12295252 0.09285913 0.09582098 0.08762737 0.13091344
  0.10733996 0.08628499 0.10416031 0.0755185 ]
 [0.02610867 0.02668229 0.02367894 0.03069979 0.3172

KeyboardInterrupt: 